# Data Warehouse Truven QA - Claim Diag

## Initialization

Just loading packages that will be used and initializing connection to GP DB.

In [1]:
import pandas as pd
import sys
import psycopg2
from tqdm import tqdm
sys.path.append('H:/uth_helpers')
from db_utils import get_dsn

In [53]:
connection = psycopg2.connect(get_dsn())
connection.autocommit = True

In [ ]:
year_df = pd.read_sql('select distinct year from dw_staging.claim_header_1_prt_truv;', con=connection)
tables = ['ccaes', 'mdcrs', 'mdcro', 'ccaeo']

c:\Users\iperez9\.conda\envs\ip_env\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


## Row Count and Claim Count

Similar to the member_enrollment_monthly table, the claim_header table has the row count should equal the number of unique claims.

For this table, we extract claim data from the s, o, and f tables from the truven schema. 


In [9]:
query = ''' drop table if exists qa_reporting.dw_truv_claim_diag_counts;
create table qa_reporting.dw_truv_claim_diag_counts
(
    calendar_year int,
    dw_row_count bigint,
    src_row_count bigint,
    row_count_diff bigint,
    row_count_percentage float,
    dw_uth_clm_id_count bigint,
    dw_src_clm_id_count bigint,
    src_clm_count bigint,
    clm_count_diff bigint,
    clm_count_percentage float,
    dw_uth_mbr_id_count bigint,
    dw_src_mbr_id_count bigint,
    src_mbr_count bigint,
    mbr_count_diff bigint,
    mbr_count_percentage float,
    date_generated date
);
'''

with connection.cursor() as cursor:
    cursor.execute(query)

In [11]:
with connection.cursor() as cursor:
    query = f'''
    insert into qa_reporting.dw_truv_claim_diag_counts
    (calendar_year, dw_row_count, dw_uth_clm_id_count, date_generated)
    select year, count(*), count(distinct uth_claim_id), current_date
    from dw_staging.claim_diag_1_prt_truv
    where diag_position = 1
    group by 1
    '''
    
    cursor.execute(query)

    query = f'''
    update qa_reporting.dw_truv_claim_diag_counts b
    set dw_src_clm_id_count = count
    from (
        select year,  count(distinct claim_id_src) as count 
        from dw_staging.claim_diag_1_prt_truv
    group by 1) a
    where a.year = b.calendar_year
    '''

    cursor.execute(query)

    query = f'''
    update qa_reporting.dw_truv_claim_diag_counts b
    set dw_uth_mbr_id_count = count
    from (
        select year, count(distinct uth_member_id) as count 
        from dw_staging.claim_diag_1_prt_truv
    group by 1) a
    where a.year = b.calendar_year
    '''

    cursor.execute(query)
    
    query = f'''
    update qa_reporting.dw_truv_claim_diag_counts b
    set dw_src_mbr_id_count = count
    from (
        select year, count(distinct member_id_src) as count 
        from dw_staging.claim_diag_1_prt_truv
    group by 1) a
    where a.year = b.calendar_year
    '''

    cursor.execute(query)

In [15]:
# Updating raw counts, but with the combine counts of all the raw tables used
# Using the counts found during the claim header qa to avoid having to redo the counts.
with connection.cursor() as cursor:
    query = '''update qa_reporting.dw_truv_claim_diag_counts a
    set src_row_count = b.src_row_count,
    row_count_diff = a.dw_row_count - b.src_row_count,
    row_count_percentage = 100. * abs(a.dw_row_count - b.src_row_count) / b.src_row_count,
    src_clm_count = b.src_clm_count,
    clm_count_diff = a.dw_uth_clm_id_count - b.src_clm_count,
    clm_count_percentage = 100. * abs(a.dw_uth_clm_id_count - b.src_clm_count) / b.src_clm_count,
    src_mbr_count = b.src_mbr_count,
    mbr_count_diff = a.dw_uth_mbr_id_count - b.src_mbr_count,
    mbr_count_percentage = 100. * abs(a.dw_uth_mbr_id_count - b.src_mbr_count) / b.src_mbr_count
    from qa_reporting.dw_truv_claim_header_counts b
    where a.calendar_year = b.calendar_year
    and b.table_src = 'ALL'
    ;
    '''

    cursor.execute(query)

In [16]:
query = '''select * from qa_reporting.dw_truv_claim_diag_counts;'''

df = pd.read_sql(query, con=connection)
df.sort_values(['clm_count_percentage'], ascending=False)

c:\Users\iperez9\.conda\envs\ip_env\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,calendar_year,dw_row_count,src_row_count,row_count_diff,row_count_percentage,dw_uth_clm_id_count,dw_src_clm_id_count,src_clm_count,clm_count_diff,clm_count_percentage,dw_uth_mbr_id_count,dw_src_mbr_id_count,src_mbr_count,mbr_count_diff,mbr_count_percentage,date_generated
6,2022,131409961,132327243,-917282,0.693192,131317636,131324138,132234183,-916547,0.693124,15125232,15125232,15154145,-28913,0.190793,2023-05-04
7,2017,291710355,293589258,-1878903,0.639977,291449291,291785855,293326352,-1877061,0.639922,22234472,22234472,22249093,-14621,0.065715,2023-05-04
1,2021,286954924,288791345,-1836421,0.635899,286763700,286772668,288599077,-1835377,0.635961,20033369,20033369,20044196,-10827,0.054016,2023-05-04
0,2020,270017353,271653875,-1636522,0.602429,269818684,269835937,271454354,-1635670,0.602558,20228996,20228996,20239042,-10046,0.049637,2023-05-04
11,2018,289404307,290789941,-1385634,0.476507,289108127,289114087,290492110,-1383983,0.476427,22537853,22537853,22547774,-9921,0.044000,2023-05-04
3,2019,298278163,299676211,-1398048,0.466520,298029554,298060919,299426746,-1397192,0.466622,21769097,21769097,21778393,-9296,0.042685,2023-05-04
9,2016,328082554,329515942,-1433388,0.434998,327772438,327892137,329186623,-1414185,0.429600,24700272,24700272,24725600,-25328,0.102436,2023-05-04
10,2015,320116332,321329053,-1212721,0.377408,319842196,319990400,321027190,-1184994,0.369126,24247419,24247419,24278257,-30838,0.127019,2023-05-04
5,2012,605974214,607373049,-1398835,0.230309,605362269,605410363,606755083,-1392814,0.229551,47476633,47476633,47495022,-18389,0.038718,2023-05-04
2,2014,504229428,505315935,-1086507,0.215015,503814645,503829765,504883546,-1068901,0.211712,39644689,39644689,39663323,-18634,0.046980,2023-05-04


## Diagnosis Codes

Here we will check if we have valid ICD Diagnosis codes using our reference tables.

In [28]:
query = '''drop table if exists qa_reporting.dw_truv_diag_counts;
select year, diag_cd, count(*) as diag_count
into qa_reporting.dw_truv_diag_counts
from dw_staging.claim_diag_1_prt_truv
group by 1,2;
'''

with connection.cursor() as cursor:
    cursor.execute(query)

In [48]:
diag_cd_df = pd.read_sql('select * from qa_reporting.dw_truv_diag_counts;', con=connection)
diag_cd_df

c:\Users\iperez9\.conda\envs\ip_env\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,year,diag_cd,diag_count
0,2020,R9081,149
1,2020,S32491D,296
2,2016,S52511K,30
3,2014,65541,58
4,2017,S9051,7
...,...,...,...
568339,2021,S46192D,1478
568340,2017,M4850XD,1402
568341,2016,T3182,1
568342,2018,S91131,3


We see that we have a lot of claims with diagnosis codes that are not in our reference list. If we take a further look at some of these codes, we see that in our reference list, there is at least one more digit missing from these codes. The missing digits helps specify the diagnosis code.

In [30]:
query = '''
select a.*
from qa_reporting.dw_truv_diag_counts a
left join reference_tables.ref_cms_icd_cm_codes b
on a.diag_cd = cd_value
where b.cd_value is null;
'''

invalid_diag_df = pd.read_sql(query, con=connection)
invalid_diag_df

c:\Users\iperez9\.conda\envs\ip_env\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,year,diag_cd,diag_count
0,2017,S199XX,85
1,2019,S0090X,5
2,2021,S199XX,121
3,2020,V890X,3
4,2017,S0282X,5
...,...,...,...
6979,2017,W1782X,1
6980,2022,O642X,7
6981,2020,V9732X,2
6982,2021,O648XX,2


Overall, the number of invalid diagnosis codes is neglible compared to the overall number of diagnosis codes in the claim_diag table.

In [31]:
invalid_diag_df.groupby('year')['diag_count'].sum()

year
2014       20
2016       13
2017    75085
2018    68611
2019    83546
2020    52658
2021    65178
2022    55465
Name: diag_count, dtype: int64

In [50]:
diag_comp_df = pd.DataFrame({'overall_diag_count': diag_cd_df.groupby('year')['diag_count'].sum(),
                            'invalid_diag_count': invalid_diag_df.groupby('year')['diag_count'].sum(),
                            'valid_diag_count': diag_cd_df.groupby('year')['diag_count'].sum() - invalid_diag_df.groupby('year')['diag_count'].sum()})
diag_comp_df.loc[diag_comp_df['valid_diag_count'].isna(),'valid_diag_count'] = diag_comp_df.loc[diag_comp_df['valid_diag_count'].isna(),'overall_diag_count']
diag_comp_df['valid_diag_count'] =  diag_comp_df['valid_diag_count'].astype(int)
diag_comp_df['invalid_to_valid_percent'] = 100. * diag_comp_df['invalid_diag_count'] / diag_comp_df['valid_diag_count']
diag_comp_df  

,overall_diag_count,invalid_diag_count,valid_diag_count,invalid_to_valid_percent
year,,,,
2011,1000763426,NaN,1000763426,NaN
2012,1042878652,NaN,1042878652,NaN
2013,858324139,NaN,858324139,NaN
2014,976199162,20.0,976199142,0.000002
2015,644045944,NaN,644045944,NaN
2016,689344947,13.0,689344934,0.000002
2017,623023827,75085.0,622948742,0.012053
2018,623176402,68611.0,623107791,0.011011
2019,653317653,83546.0,653234107,0.012790


In [51]:
invalid_diag_df.groupby('year').max()

,diag_cd,diag_count
year,,
2014,7070,6
2016,Z2839,3
2017,Y33XXX,6597
2018,Y33XX,4258
2019,Y33XXX,4398
2020,Y31XXX,3413
2021,Y32XX,4004
2022,Z2839,13608


## Diagnosis Code Position

In [ ]:
query = '''drop table if exists qa_reporting.dw_truv_diag_position;
select year, diag_position, count(distinct uth_claim_id) as claim_count
into qa_reporting.dw_truv_diag_position
from dw_staging.claim_diag_1_prt_truv
group by 1,2;
'''
with connection.cursor() as cursor:
    cursor.execute(query)


In [54]:
diag_position_df = pd.read_sql('select * from qa_reporting.dw_truv_diag_position;', con=connection)
diag_position_df.sort_values(['year', 'diag_position'])

,year,diag_position,claim_count
1,2011,1,603799006
9,2011,2,224226728
47,2011,3,113303205
53,2011,4,52552455
31,2011,5,5548746
37,2012,1,605362269
39,2012,2,244050645
59,2012,3,127394146
10,2012,4,58855102
18,2012,5,5903502


In [73]:
diag_position_df['diag_position'].unique()

array([4, 1, 2, 3, 5], dtype=int64)

Checking if the counts for the diag_position are correct. The higher the diag_position is, the less counts there should be. If we sort the counts of the diag_position by year and assign their order, this value should match with the diag_position value.

In [70]:
diag_position_df['row_rank'] = diag_position_df.sort_values(['year', 'claim_count'], ascending=[True, False]).groupby(['year']).cumcount()+1
diag_position_df['position_check'] = diag_position_df['row_rank'] == diag_position_df['diag_position']
diag_position_df[~diag_position_df['position_check']]

,year,diag_position,claim_count,row_rank,position_check


## ICD Version

In [34]:
query = '''
select year, icd_version, count(distinct uth_claim_id) as claim_count
into qa_reporting.dw_truv_icd_version_count
from dw_staging.claim_diag_1_prt_truv
group by 1,2;
'''
with connection.cursor() as cursor:
    cursor.execute(query)

In [36]:
icd_version_df = pd.read_sql('select * from qa_reporting.dw_truv_icd_version_count;', con=connection)
icd_version_df.sort_values(['year', 'icd_version'])

c:\Users\iperez9\.conda\envs\ip_env\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,year,icd_version,claim_count
35,2011,0,1802
30,2011,9,5154
31,2011,None,603846894
34,2012,0,2169
20,2012,9,6408
21,2012,None,605415909
3,2013,0,4278
7,2013,9,13140
9,2013,None,486609012
12,2014,0,12835
